## 1. 内存优化探讨
- 通常会从设备分级、Bitmap 优化和内存泄漏这三个方面入手

### 1.1 设备分级
- 内存优化首先需要根据设备环境来综合考虑。其实我们可以让高端设备使用更多的内存，做到针对设备性能的好坏使用不同的内存分配和回收策略。
- 这需要一个比较好的架构设计
  1. 设备分级
     - 使用类似 device-year-class 的策略对设备分级，对于低端机用户可以关闭复杂的动画，或者是某些功能；
     - 使用 565 格式的图片，使用更小的缓存内存等策略
     ```
     if (year >= 2013) {
        // Do advanced animation
     } else if (year >= 2010) {
         // Do simple animation
     } else {
         // Phone too slow, don't do any animations
     }
     ```
  2. 缓存管理
     - 需要有一套统一的缓存管理机制，可以适当地使用内存；当“系统有难”时，也要义不容辞地归还。
     - 可以使用 OnTrimMemory 回调，根据不同的状态决定释放多少内存。
  3. 进程模型
     - 一个空的进程也会占用 10MB 的内存，而有些应用启动就有十几个进程，甚至有些应用已经从双进程保活升级到四进程保活
     - 减少应用启动的进程数、减少常驻进程、有节操的保活，对低端机内存优化非常重要
  4. 安装包大小
     - 安装包中的代码、资源、图片以及 so 库的体积，跟它们占用的内存有很大的关系
     - 一个 80MB 的应用很难在 512MB 内存的手机上流畅运行
     - 这种情况我们需要考虑针对低端机用户推出 4MB 的轻量版本，例如 Facebook Lite、今日头条极速版都是这个思路。
     ![image](memory2_page1.png)

### 1.2 Bitmap 优化
1. 统一图片库
- 图片内存优化的前提是收拢图片的调用，这样就可以做整体的控制策略
  - 例如低端机使用 565 格式、更加严格的缩放算法，可以使用 Glide、Fresco 或者采取自研都可以
  - 需要进一步将所有 Bitmap.createBitmap、BitmapFactory 相关的接口也一并收拢

2. 统一监控
- 大图片监控
  - 注意某张图片内存占用是否过大，例如长宽远远大于 view 甚至屏幕的长宽
  - 检测到不合规的图片使用，应该立即弹出对话框提示图片所在的 Activity 和堆栈，可以更快的定位问题
  - 如果是预发和线上环境，则可以上报后台去做记录，然后分析优化

- 重复图片
  - 重复图片指的是 Bitmap 的像素数据完全一致，但是有多个不同的对象存在
  
- 图片总内存
  - 在线上就可以按不同的系统、屏幕分辨率等维度去分析图片内存的占用的内存，在线上就可以按不同的系统、屏幕分辨率等维度去分析图片内存的占用情况
  - **在 OOM 崩溃的时候，也可以把图片占用的总内存、Top N 图片的内存都写到崩溃日志中，帮助我们排查问题**

### 1.3 内存泄漏
- 内存泄漏主要分两种情况，一种是同一个对象泄漏，还有一种情况更加糟糕，就是每次都会泄漏新的对象，可能会出现几百上千个无用的对象

1. Java 内存泄漏
  - 建立类似 LeakCanary 自动化检测方案，至少做到 Activity 和 Fragment 的泄漏检测
  - 内存泄漏监控放到线上并不容易，可以对生成的 Hprof 内存快照文件做一些优化，裁剪大部分图片对应的 byte 数组减少文件大小
    - 比如一个 100MB 的文件裁剪后一般只剩下 30MB 左右，使用 7zip 压缩最后小于 10MB，增加了文件上传的成功率
    
2. OOM 监控
  - 美团有一个 Android 内存泄露自动化链路分析组件 [Probe](https://static001.geekbang.org/con/19/pdf/593bc30c21689.pdf)
  - 在发送 OOM 时候生成 Hprof 内存快照，然后通过单独的进程对这个文件做进一步分析（线上存在风险，可能会导致二次崩溃；有些机型生成快照会耗时几分钟，体验不好；部分 OOM 是因为虚拟内存不足，这需要具体分析；）
  
3. Native 内存泄漏监控
  - 参考文档：[《微信 Android 终端内存优化实践》](https://mp.weixin.qq.com/s/KtGfi5th-4YHOZsEmTOsjg?)中，微信也做了一些其他方案上面的尝试

4. 针对无法重编 so 的情况
  - 通过 GCC 的 "-finstrument-functions" 参数给所有函数插装，桩中模拟调用栈入栈出栈操作；
  - 通过 ld 的 "–wrap" 参数拦截内存分配和释放函数，重定向到我们自己的实现后记录分配的内存地址、大小、来源 so 以及插桩记录的调用栈此刻的内容，定期扫描分配与释放是否配对，对于不配对的分配输出我们记录的信息
  
- 开发过程中内存泄漏排查可以使用 Androd Profile和 MAT 工具配合使用，而日常监控关键是成体系化，做到及时发现问题

---

## 2. 内存监控
### 2.1 采集方式
- 用户在前台的时候，可以每 5 分钟采集一次 PSS、Java 堆、图片总内存
- 建议通过采样只统计部分用户，需要注意的是要按照用户抽样，而不是按次抽样
  - 简单来说一个用户如果命中采集，那么在一天内都要持续采集数据

### 2.2 计算指标
> 内存异常率：可以反映内存占用的异常情况，如果出现新的内存使用不当或内存泄漏的场景，这个指标会有所上涨。其中 PSS 的值可以通过 Debug.MemoryInfo 拿到
```
内存 UV 异常率 = PSS 超过 400MB 的 UV / 采集 UV
```

> 触顶率：可以反映 Java 内存的使用情况，如果超过 85% 最大堆限制，GC 会变得更加频繁，容易造成 OOM 和卡顿。
```
内存 UV 触顶率 = Java 堆占用超过最大堆限制的 85% 的 UV / 采集 UV
```

- 其中是否触顶可以通过下面的方法计算得到
```
long javaMax = runtime.maxMemory();
long javaTotal = runtime.totalMemory();
long javaUsed = javaTotal - runtime.freeMemory();
// Java 内存使用超过最大限制的 85%
float proportion = (float) javaUsed / javaMax;
```

- 一般客户端只上报数据，所有计算都在后台处理，这样可以做到灵活多变。后台还可以计算平均 PSS、平均 Java 内存、**平均图片占**用这些指标，它们可以反映内存的平均情况。
![image](memory2_page2.png)

### 2.3 GC 监控
- 在实验室或者内部试用环境，可以通过 Debug.startAllocCounting 来监控 Java 内存分配和 GC 的情况，需要注意的是这个选项对性能有一定的影响，虽然目前还可以使用，但已经被 Android 标记为 deprecated
```
// 通过监控，可以拿到内存分配的次数和大小，以及 GC 发起次数等信息
long allocCount = Debug.getGlobalAllocCount();
long allocSize = Debug.getGlobalAllocSize();
long gcCount = Debug.getGlobalGcInvocationCount();
```

- 在 Android 6.0 之后系统可以拿到更加精准的 GC 信息
```
// 运行的 GC 次数
Debug.getRuntimeStat("art.gc.gc-count");
// GC 使用的总耗时，单位是毫秒
Debug.getRuntimeStat("art.gc.gc-time");
// 阻塞式 GC 的次数
Debug.getRuntimeStat("art.gc.blocking-gc-count");
// 阻塞式 GC 的总耗时
Debug.getRuntimeStat("art.gc.blocking-gc-time");
```

---

## 总结
- 在具体进行内容优化前，首先要问清楚自己几个问题，比如我们要优化到什么目标、内存对我们造成了多少异常和卡顿

---

## 课后作业
- Hprof 文件裁剪和重复图片监控，是很多应用目前都没有做的
- 尝试使用 HAHA 库快速判断内存中是否存在重复的图片，并且将这些重复图片的 PNG、堆栈等信息输出。最终的实现可以通过向 Sample 发送 Pull Request

---